<a href="https://colab.research.google.com/github/Premkumar-DataEngineer/DataEngineering/blob/main/Project_bank_dataclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,199 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,576 kB]
Get:14 http://arch

In [4]:
!export JAVA_HOME="/usr/lib/jvm/java-11-openjdk-amd64"

In [5]:
%pip install findspark

In [6]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Colab").getOrCreate()

In [7]:
from pyspark import SparkContext

In [9]:
sc = spark.sparkContext

In [10]:
!ls

sample_data


In [11]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/refs/heads/master/bank_prospects.csv

--2025-08-27 18:50:41--  https://raw.githubusercontent.com/futurexskill/bigdata/refs/heads/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2025-08-27 18:50:42 (4.34 MB/s) - ‘bank_prospects.csv’ saved [306/306]



In [12]:
!ls

bank_prospects.csv  sample_data


In [13]:
bankrdd=sc.textFile("bank_prospects.csv")

In [14]:
bankrdd.collect()

['Age,Salary,Gender,Country,Purchased',
 '18,20000,Male,Germany,N',
 '19,22000,Female,France,N',
 '20,24000,Female,England,N',
 '21,,Male,England,N',
 '22,50000,Male,France,Y',
 '23,35000,Female,England,N',
 '24,,Male,Germany,N',
 '25,32000,Female,France,Y',
 ',35000,Male,Germany,N',
 '27,37000,Female,France,N',
 '27,37000,Female,unknown,N']

In [15]:
bankdf=spark.read.csv("bank_prospects.csv",header=True,inferSchema=True)

In [16]:
bankdf.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  NULL|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  NULL|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|NULL| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



In [17]:
bankdf.agg({'Age':'avg'}).show()

+--------+
|avg(Age)|
+--------+
|    22.6|
+--------+



In [18]:
bankdf.withColumn('newAge':avg(bankdf['Age'])).show()

SyntaxError: invalid syntax (ipython-input-1898217755.py, line 1)

In [19]:
bankdf.select(avg('Age')).show()

NameError: name 'avg' is not defined

In [29]:
from pyspark.sql.functions import avg, col, when

In [21]:
bankdf.select(avg('Age')).show()

+--------+
|avg(Age)|
+--------+
|    22.6|
+--------+



In [42]:
avg_age, avg_salary=bankdf.select(avg('Age'),avg('Salary')).collect()[0]
bankdffillna=bankdf.fillna(avg_age,['Age'])
#bankdffillna=bankdffillna.withColumn("Salary", when(col("Salary").isNull(), avg_salary).otherwise(col("Salary")))
#bankdffillna=bankdffillna.replace("NULL",None, subset=["Salary"])
bankdffillna=bankdffillna.fillna(avg_salary,['Salary'])
bankdffillna.show()


+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21| 32444|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 24| 32444|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 22| 35000|  Male|Germany|        N|
| 27| 37000|Female| France|        N|
| 27| 37000|Female|unknown|        N|
+---+------+------+-------+---------+



In [43]:
bankdffillna.show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21| 32444|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 24| 32444|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 22| 35000|  Male|Germany|        N|
| 27| 37000|Female| France|        N|
| 27| 37000|Female|unknown|        N|
+---+------+------+-------+---------+



In [44]:
bankdffillna.dropna()
bankdffillna.show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21| 32444|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 24| 32444|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 22| 35000|  Male|Germany|        N|
| 27| 37000|Female| France|        N|
| 27| 37000|Female|unknown|        N|
+---+------+------+-------+---------+



In [46]:
bankdffillna=bankdffillna.filter(bankdffillna['Country']!="unknown").show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21| 32444|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 24| 32444|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 22| 35000|  Male|Germany|        N|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+

